# Project "Revenu des adults" (Adult income)

## <a name="setup"></a> Configuration générale

In [1]:
import os

print ('Projet : %s' %os.getcwd())

workingDir = '/Volumes/Data/data_analysis/EILV/'
os.chdir(workingDir)

projectDir = '%s/projects/adult-income/' %os.getcwd()

Projet : /Volumes/Data/data_analysis/EILV/projects/adult-income


In [3]:
import os
# workingDir = '.'
# os.chdir(workingDir)

conf_file = '%sconfig-adult-income.yaml' %projectDir
Sample_Size = 10000

print ('Fichier de configuration: %s' %conf_file)


Fichier de configuration: /Volumes/Data/data_analysis/EILV/projects/adult-income/config-adult-income.yaml


### Importer les paquets nécessaires et configurer les paramètres de l'environnement

In [4]:
import pandas as pd
import numpy as np
import os
#os.chdir(workingDir)
import collections
import matplotlib
import io
import sys
import operator

import nbformat as nbf
from IPython.core.display import HTML
from IPython.display import display
from ipywidgets import interact, interactive,fixed
from IPython.display import Javascript, display,HTML
from ipywidgets import widgets, VBox
import ipywidgets
import IPython
from IPython.display import clear_output
import scipy.stats as stats
from statsmodels.graphics.mosaicplot import mosaic
import statsmodels.api as sm
from statsmodels.formula.api import ols
import os
import errno
import seaborn as sns
from string import Template
from functools import partial
from collections import OrderedDict

# Utility Classes
from utilities.ConfUtility import * 
from utilities.ReportGeneration import *
from utilities.UniVarAnalytics import *
from utilities.MultiVarAnalytics import *

%matplotlib inline

# Plotly
import plotly.offline as py
from plotly import tools as tls
py.init_notebook_mode(connected=True) #initiate the Plotly Notebook mode

#DEBUG=0

font={'family':'normal','weight':'normal','size':8}
matplotlib.rc('font',**font)
matplotlib.rcParams['figure.figsize'] = (12.0, 5.0)
matplotlib.rc('xtick', labelsize=9) 
matplotlib.rc('ytick', labelsize=9)
matplotlib.rc('axes', labelsize=10)
matplotlib.rc('axes', titlesize=10)
sns.set_style('whitegrid')

print ('Importation finie !')

Importation finie !


## <a name="read and summarize"></a> Lecture récapitulative des données

### Lire les données et déduire les types de colonnes

In [5]:
conf_dict = ConfUtility.parse_yaml(conf_file)

# Read in data from local file or SQL server
if 'DataSource' not in conf_dict:
    df=pd.read_csv('%s%s' %(projectDir,conf_dict['DataFilePath'][0]), skipinitialspace=True)
else:
    import pyodbc
    cnxn = pyodbc.connect('driver=MySQL ODBC 3.51 Driver;server={};database={};Uid={};Pwd={}'.format(
            conf_dict['Server'], conf_dict['Database'],conf_dict['Username'],conf_dict['Password']))
    df = pd.read_sql(conf_dict['Query'],cnxn)

# Making sure that we are not reading any extra column
df = df[[each for each in df.columns if 'Unnamed' not in each]]

# Sampling Data if data size is larger than 10k
df0 = df # df0 is the unsampled data. Will be used in data exploration and analysis where sampling is not needed
         # However, keep in mind that your final report will always be based on the sampled data. 
if Sample_Size < df.shape[0]:
    df = df.sample(Sample_Size)

# change float data types
if 'FloatDataTypes' in conf_dict:   
    for col_name in conf_dict['FloatDataTypes']:
        df[col_name] = df[col_name].astype(float)      
        
# Getting the list of categorical columns if it was not there in the yaml file
if 'CategoricalColumns' not in conf_dict:
    conf_dict['CategoricalColumns'] = list(set(list(df.select_dtypes(exclude=[np.number]).columns)))

# Getting the list of numerical columns if it was not there in the yaml file
if 'NumericalColumns' not in conf_dict:
    conf_dict['NumericalColumns'] = list(df.select_dtypes(include=[np.number]).columns)    

# Exclude columns that we do not need
if 'ColumnsToExclude' in conf_dict:
    conf_dict['CategoricalColumns'] = list(set(conf_dict['CategoricalColumns'])-set(conf_dict['ColumnsToExclude']))
    conf_dict['NumericalColumns'] = list(set(conf_dict['NumericalColumns'])-set(conf_dict['ColumnsToExclude']))

# Ordering the categorical variables according to the number of unique categories
filtered_cat_columns = []
temp_dict = {}
for cat_var in conf_dict['CategoricalColumns']:
    temp_dict[cat_var] = len(np.unique(df[cat_var]))
sorted_x = sorted(temp_dict.items(), key=operator.itemgetter(0), reverse=True)
conf_dict['CategoricalColumns'] = [x for (x,y) in sorted_x]

ConfUtility.dict_to_htmllist(conf_dict,['Target','CategoricalColumns','NumericalColumns'])

### Lister les n-premières lignes (par défault, n=5)

In [7]:
def custom_head(df,NoOfRows):
    return HTML(df.head(NoOfRows).style.set_table_attributes("class='table'").render())
i = interact(custom_head,df=fixed(df0), NoOfRows=ipywidgets.IntSlider(min=0, max=1000, step=1, \
                                                                     value=5, description='Nombre de ligne'))

interactive(children=(IntSlider(value=5, description='Nombre de ligne', max=1000), Output()), _dom_classes=('w…

### Afficher les dimensions des données (lignes, colonnes)

In [8]:
print ('{} lignes | {} colonnes'.format(df0.shape[0],df0.shape[1]))

32561 lignes | 15 colonnes


### Imprimer les noms des variables (colonnes)

In [9]:
col_names = ','.join(each for each in list(df.columns))
print("Les variables sont :" + col_names)

Les variables sont :age,workclass,fnlwgt,education,educationnum,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,nativecountry,label_IsOver50K


### Afficher les types de colonnes

In [10]:
print("Types des variables:")
df.dtypes

Types des variables:


age                 int64
workclass          object
fnlwgt              int64
education          object
educationnum        int64
maritalstatus      object
occupation         object
relationship       object
race               object
sex                object
capitalgain         int64
capitalloss         int64
hoursperweek        int64
nativecountry      object
label_IsOver50K     int64
dtype: object

## <a name="individual variable"></a>Extraire les statistiques descriptives de chaque colonne

In [11]:
def num_missing(x):
    return len(x.index)-x.count()

def num_unique(x):
    return len(np.unique(x))

temp_df = df0.describe().T
missing_df = pd.DataFrame(df0.apply(num_missing, axis=0)) 
missing_df.columns = ['données manquantes']
unq_df = pd.DataFrame(df0.apply(num_unique, axis=0))
unq_df.columns = ['unique']
types_df = pd.DataFrame(df0.dtypes)
types_df.columns = ['Type de données']

### Afficher les statistiques descriptives des variables quantitatives

In [12]:
summary_df = temp_df.join(missing_df).join(unq_df).join(types_df)
summary_df

,count,mean,std,min,25%,50%,75%,max,données manquantes,unique,Type de données
age,32561.0,38.581647,13.640433,17.0,28.0,37.0,48.0,90.0,0,73,int64
fnlwgt,32561.0,189778.366512,105549.977697,12285.0,117827.0,178356.0,237051.0,1484705.0,0,21648,int64
educationnum,32561.0,10.080679,2.572720,1.0,9.0,10.0,12.0,16.0,0,16,int64
capitalgain,32561.0,1077.648844,7385.292085,0.0,0.0,0.0,0.0,99999.0,0,119,int64
capitalloss,32561.0,87.303830,402.960219,0.0,0.0,0.0,0.0,4356.0,0,92,int64
hoursperweek,32561.0,40.437456,12.347429,1.0,40.0,40.0,45.0,99.0,0,94,int64
label_IsOver50K,32561.0,0.240810,0.427581,0.0,0.0,0.0,0.0,1.0,0,2,int64


### Afficher les statistiques descriptives des variables qualitatives

In [13]:
col_names = list(types_df.index) #Get all col names
num_cols = len(col_names)
index = range(num_cols)
cat_index = []
for i in index: #Find the indices of columns in Categorical columns
    if col_names[i] in conf_dict['CategoricalColumns']:
        cat_index.append(i)
summary_df_cat = missing_df.join(unq_df).join(types_df.iloc[cat_index], how='inner') #Only summarize categorical columns
summary_df_cat

,données manquantes,unique,Type de données
workclass,0,9,object
education,0,16,object
maritalstatus,0,7,object
occupation,0,15,object
relationship,0,6,object
race,0,5,object
sex,0,2,object
nativecountry,0,42,object
label_IsOver50K,0,2,int64


## <a name="individual variables"></a>Explorer des variables individuelles

### Explorer la variable cible

In [14]:
w1_value,w2_value,w3_value = '','',''
w1, w2, w3, w4 = None, None, None, None
w1 = widgets.Dropdown(
        options=[conf_dict['Target']],
        value=conf_dict['Target'],
        description='Variable cible:',
    )

if conf_dict['Target'] in conf_dict['CategoricalColumns']: 
    i = interactive(TargetAnalytics.custom_barplot, df=fixed(df), col1=w1)
else:
    i = interactive(NumericAnalytics.custom_barplot, df=fixed(df), col1=w1)
display(i)

interactive(children=(Dropdown(description='Variable cible:', options=('label_IsOver50K',), value='label_IsOve…

### Explorer des variables numériques individuelles et tester la normalité (sur des données échantillonnées)

In [15]:
w1_value, w2_value, w3_value = '', '', ''
w1, w2, w3, w4 = None, None, None, None
w1 = widgets.Dropdown(
        options=conf_dict['NumericalColumns'],
        value=conf_dict['NumericalColumns'][0],
        description='Numeric Variable:',
    )

i = interactive(NumericAnalytics.custom_barplot, df=fixed(df), col1=w1)
display(i)

interactive(children=(Dropdown(description='Numeric Variable:', options=('age', 'fnlwgt', 'educationnum', 'cap…

### Explorer des variables qualitatives individuelles (triées par fréquences)

In [16]:
w1_value, w2_value, w3_value = '', '', ''
w1, w2, w3, w4 = None, None, None, None
w1 = widgets.Dropdown(
    options = conf_dict['CategoricalColumns'],
    value = conf_dict['CategoricalColumns'][0],
    description = 'Categorical Variable:',
)
# w1.observe (handle_change,'value')
i = interactive(CategoricAnalytics.custom_barplot, df=fixed(df), col1=w1)

display(i)

interactive(children=(Dropdown(description='Categorical Variable:', options=('workclass', 'sex', 'relationship…

## <a name="multiple variables"></a>Explorer les interactions entre les variables

### <a name="rank variables"></a>Classer les variables en fonction des relations linéaires avec la variable de référence (sur les données échantillonnées)

In [17]:
cols_list = [conf_dict['Target']] + conf_dict['NumericalColumns'] + conf_dict['CategoricalColumns'] #Make target the default reference variable
cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
w1 = widgets.Dropdown(    
    options=cols_list,
    value=cols_list[0],
    description='Ref Var:'
)
w2 = ipywidgets.Text(value="5", description='Top Num Vars:')
w3 = ipywidgets.Text(value="5", description='Top Cat Vars:')
i = interactive(InteractionAnalytics.rank_associations, df=fixed(df), conf_dict=fixed(conf_dict), col1=w1, col2=w2, col3=w3)
display(i)

interactive(children=(Dropdown(description='Ref Var:', options=('label_IsOver50K', 'age', 'fnlwgt', 'education…

### <a name="two categorical"></a>Explorer les interactions entre les variables qualitatives

In [18]:
w1, w2, w3, w4 = None, None, None, None

if conf_dict['Target'] in conf_dict['CategoricalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['CategoricalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['CategoricalColumns']
    
w1 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[0],
    description='Var Qual 1:'
)
w2 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[1],
    description='Var Qual 2:'
)
i = interactive(InteractionAnalytics.categorical_relations, df=fixed(df), col1=w1, col2=w2)
display(i)

interactive(children=(Dropdown(description='Var Qual 1:', options=('label_IsOver50K', 'workclass', 'sex', 'rel…

### <a name="two numerical"></a>Explorer les interactions entre les variables quantitatives (sur les données échantillonnées)

In [19]:
w1, w2, w3, w4 = None, None, None, None

if conf_dict['Target'] in conf_dict['NumericalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['NumericalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['NumericalColumns']
w1 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[0],
    description='Var quanti 1:'
)
w2 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[1],
    description='Var quanti 2:'
)
i = interactive(InteractionAnalytics.numerical_relations, df=fixed(df), col1=w1, col2=w2)
display(i)

interactive(children=(Dropdown(description='Var quanti 1:', options=('age', 'fnlwgt', 'educationnum', 'capital…

### Explorer la matrice de corrélation entre les variables quantitatives

In [20]:
w1, w2, w3, w4 = None, None, None, None
w1 = widgets.Dropdown(
    options=['pearson','kendall','spearman'],
    value='pearson',
    description='Methode:'
)
i = interactive(InteractionAnalytics.numerical_correlation, df=fixed(df), conf_dict=fixed(conf_dict),\
                                         col1=w1)

display(i)

interactive(children=(Dropdown(description='Methode:', options=('pearson', 'kendall', 'spearman'), value='pear…

### <a name="numerical and categorical"></a>Explorer les interactions entre les variables quantitative et qualitative

In [21]:
w1, w2, w3, w4 = None, None, None, None

if conf_dict['Target'] in conf_dict['NumericalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['NumericalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['NumericalColumns']
    
w1 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[0],
    description='Quantitative:'
)

if conf_dict['Target'] in conf_dict['CategoricalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['CategoricalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['CategoricalColumns']
    
w2 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[0],
    description='Qualitative:'
)
i = interactive(InteractionAnalytics.nc_relation, df=fixed(df), \
                                                conf_dict=fixed(conf_dict), col1=w1, col2=w2, \
                                                col3=fixed(w3))

display(i)

interactive(children=(Dropdown(description='Quantitative:', options=('age', 'fnlwgt', 'educationnum', 'capital…

### <a name="two numerical and categorical"></a>Explorer les interactions entre deux variables quantitatives et une variable qualitatives (sur des données échantillonnées)

In [22]:
w1, w2, w3, w4 = None, None, None, None

if conf_dict['Target'] in conf_dict['NumericalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['NumericalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['NumericalColumns']
    
w1 = widgets.Dropdown(
    options = cols_list,
    value = cols_list[0],
    description = 'Var Qttv 1:'
)
w2 = widgets.Dropdown(
    options = cols_list,
    value = cols_list[1],
    description = 'Var Qttv 2:'
)

if conf_dict['Target'] in conf_dict['CategoricalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['CategoricalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['CategoricalColumns']
    
w3 = widgets.Dropdown(
    options = cols_list,
    value = cols_list[0],
    description = 'Legend Cat Var:'
)
i = interactive(InteractionAnalytics.nnc_relation, df=fixed(df),conf_dict=fixed(conf_dict), col1=w1, col2=w2, col3=w3)
display(i)

interactive(children=(Dropdown(description='Var Qttv 1:', options=('age', 'fnlwgt', 'educationnum', 'capitalga…

## <a name="pca"></a>Visualiser les données quantitatives par analyse en composantes principales (sur les données échantillonnées)

### Projection 2D par ACP

In [23]:
num_numeric = len(conf_dict['NumericalColumns'])
if  num_numeric > 3:
    w1, w2, w3, w4, w5 = None, None, None, None, None
    if conf_dict['Target'] in conf_dict['CategoricalColumns']:
        cols_list = [conf_dict['Target']] + conf_dict['CategoricalColumns'] #Make target the default reference variable
        cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
    else:
        cols_list = conf_dict['CategoricalColumns']
    w1 = widgets.Dropdown(
        options = cols_list,
        value = cols_list[0],
        description = 'Legend:',
        width = 10
    )
    w2 = widgets.Dropdown(
        options = [str(x) for x in np.arange(1,num_numeric+1)],
        value = '1',
        width = 1,
        description='PC at X-Axis:'
    )
    w3 = widgets.Dropdown(
        options = [str(x) for x in np.arange(1,num_numeric+1)],
        value = '2',
        description = 'PC at Y-Axis:'
    )
    i = interactive(InteractionAnalytics.numerical_pca, df=fixed(df), conf_dict=fixed(conf_dict), col1=w1, col2=w2, col3=w3)

    
    display(i)

interactive(children=(Dropdown(description='Legend:', options=('label_IsOver50K', 'workclass', 'sex', 'relatio…

### Projection 3D par ACP

In [24]:
if len(conf_dict['NumericalColumns']) > 3:
    if conf_dict['Target'] in conf_dict['CategoricalColumns']:
        cols_list = [conf_dict['Target']] + conf_dict['CategoricalColumns'] #Make target the default reference variable
        cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
    else:
        cols_list = conf_dict['CategoricalColumns']
    w1, w2, w3, w4 = None, None, None, None
    w1 = widgets.Dropdown(
        options=cols_list,
        value=cols_list[0],
        description='Legend:'
    )
    w2 = ipywidgets.IntSlider(min=-180, max=180, step=5, value=30, description='Angle')
    i = interactive(InteractionAnalytics.pca_3d, df=fixed(df), conf_dict=fixed(conf_dict), col1=w1, col2=w2, col3=fixed(w3))
    display(i)

interactive(children=(Dropdown(description='Legend:', options=('label_IsOver50K', 'workclass', 'sex', 'relatio…

## <a name="show hide codes"></a>Afficher/Cacher les codes sources

In [28]:
display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
    .widget-text { min-width: 60ex !important; }
</style>'''))

#Toggle Code
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();

 } else {
 $('div.input').show();

 }
 code_show = !code_show
} 
//$( document ).ready(code_toggle);//commenting code disabling by default
</script>
<form action = "javascript:code_toggle()"><input type="submit" value="Afficher/Cacher"></form>''')